## Week 4 Lab Manual
### Foundations of Deep Learning & AI Functionality

**Instructor Note**: This lab manual provides the aim, code, and explanation for each practical task. Focus on the architectural patterns and the transition from theoretical concepts to functional AI implementations.

---

# Week 4: Training Paradigms & Prompt Engineering
## Optimizing Model Performance with Gemini

###  Weekly Table of Contents
1. [LLM-Driven Code Optimization (C++ Performance)](#-Lab-4.1:-LLM-Driven-Code-Optimization-(C++-Performance))
2. [Advanced Code Optimizer (System Prompting)](#-Lab-4.2:-Advanced-Code-Optimizer-(System-Prompting))
- Cross-Platform Compilation Logic
- System Prompting for Performance
- Real-time Execution Benchmarking

###  Learning Objectives
Effective prompting is the difference between a generic response and a state-of-the-art solution. This week, we explore advanced prompting techniques and the training history of LLMs. You will learn:
1.  **Transfer Learning**: How models are pre-trained on the internet and fine-tuned for tasks.
2.  **System Prompting**: Crafting personas and constraints for Gemini 1.5 Flash.
3.  **Few-Shot CoT**: Using Chain-of-Thought prompting to solve complex logic.
4.  **Structured Parsers**: Using LangChain to extract JSON and code.
5.  **Sampling**: Understanding Temperature, Top-P, and Top-K.

---

In [ ]:
# 📦 WEEK 4 INITIALIZATION
import os
import json
import subprocess
import platform
import io
import sys
import time
from dotenv import load_dotenv
import google.generativeai as genai
import ollama
from IPython.display import Markdown, display
import gradio as gr

# LANGCHAIN PROMPTING & PARSING
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv(override=True)
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') or os.getenv('GOOGLE_API_KEY')
HF_TOKEN = os.getenv('HF_TOKEN')

# Configure Gemini
if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
else:
    print("⚠️ WARNING: GEMINI_API_KEY not found.")

# Global Model Names
GEMINI_MODEL = 'gemini-1.5-flash'
LOCAL_MODEL = 'gemma2:2b'

# Define Sample Algorithms for the exercises
pi_calculation = """
import time
def calculate_pi(n_terms):
    start = time.time()
    pi = 0
    for i in range(n_terms):
        pi += 4 * ((-1)**i) / (2*i + 1)
    end = time.time()
    print(f"PI: {pi:.10f}")
    print(f"Time: {end - start:.6f}s")

calculate_pi(1000000)
"""

python_complex = """
import time
import random

def max_subarray_sum(n, seed, min_val, max_val):
    random.seed(seed)
    random_numbers = [random.randint(min_val, max_val) for _ in range(n)]
    
    max_sum = float('-inf')
    current_sum = 0
    for x in random_numbers:
        current_sum = max(x, current_sum + x)
        max_sum = max(max_sum, current_sum)
    return max_sum

def run_test():
    start = time.time()
    total = 0
    for i in range(20):
        total += max_subarray_sum(10000, 42 + i, -10, 10)
    end = time.time()
    print(f"Total Max Sum: {total}")
    print(f"Time: {end - start:.6f}s")

run_test()
"""

print(f"✅ Week 4 Ready: Cloud ({GEMINI_MODEL}) and Local ({LOCAL_MODEL})")

##  Lab 4.1: LLM-Driven Code Optimization (C++ Performance)
**Aim**: To leverage Gemini 1.5 Flash's code generation capabilities to convert Python algorithms into high-performance, cross-platform C++ code and benchmark the execution speed improvement.

**Explanation**:
This lab demonstrates a real-world application of LLMs in software engineering:
1.  **Cross-Platform Logic**: The script automatically detects the host OS (Windows/Linux/Mac) and identifies available C++ compilers (cl.exe, g++, clang++).
2.  **System Prompting**: We use a highly specific system prompt to force the model into "Optimizer Mode," prioritizing speed and exact numerical parity with Python.
3.  **Dynamic Compilation**: The notebook compiles the generated C++ on-the-fly, runs it, and compares the output with the original Python version to ensure correctness.

In [ ]:
# system message for code generation

system_message = "You are an assistant that reimplements Python code in high performance C++ for cross-platform compatibility. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."

In [ ]:
# prompt helper functions

def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
# file output utility

def write_output(cpp, filename="optimized.cpp"):
    """Write C++ code to file, cleaning up code blocks"""
    code = cpp.replace("```cpp","").replace("```","")
    with open(filename, "w") as f:
        f.write(code)

In [ ]:
# Gemini optimization function

def optimize_gemini(python):
    """Generate C++ code using Gemini 1.5 Flash"""
    model = genai.GenerativeModel(GEMINI_MODEL)
    
    prompt = f"{system_message}\n\n{user_prompt_for(python)}"
    
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=0.1,
            max_output_tokens=2000,
        )
    )
    
    reply = response.text
    write_output(reply)
    return reply

def optimize_ollama_gemma(python):
    """Generate C++ code using local Gemma via Ollama"""
    print(f"Generating C++ with Ollama ({LOCAL_MODEL})...")
    prompt = f"{system_message}\n\n{user_prompt_for(python)}"
    try:
        response = ollama.generate(model=LOCAL_MODEL, prompt=prompt)
        reply = response['response']
        write_output(reply)
        print(reply)
        return reply
    except Exception as e:
        print(f"Ollama Error: {e}")
        return ""


In [ ]:
# Cross-platform C++ compilation utilities

VISUAL_STUDIO_2022_TOOLS = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\Common7\\Tools\\VsDevCmd.bat"
VISUAL_STUDIO_2019_TOOLS = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"

simple_cpp_test = """
#include <iostream>

int main() {
    std::cout << "Hello";
    return 0;
}
"""

def run_cmd(command_to_run):
    """Execute a command and return its output"""
    try:
        run_result = subprocess.run(command_to_run, check=True, text=True, capture_output=True)
        return run_result.stdout if run_result.stdout else "SUCCESS"
    except:
        return ""

def c_compiler_cmd(filename_base):
    """Auto-detect C++ compiler and return compilation command"""
    my_platform = platform.system()
    my_compiler = []

    try:
        with open("simple.cpp", "w") as f:
            f.write(simple_cpp_test)
            
        if my_platform == "Windows":
            # Try Visual Studio 2022
            if os.path.isfile(VISUAL_STUDIO_2022_TOOLS):
                if os.path.isfile("./simple.exe"):
                    os.remove("./simple.exe")
                compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", "simple.cpp"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple.exe"]) == "Hello":
                        my_compiler = ["Windows", "Visual Studio 2022", ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
        
            # Try Visual Studio 2019
            if not my_compiler:
                if os.path.isfile(VISUAL_STUDIO_2019_TOOLS):
                    if os.path.isfile("./simple.exe"):
                        os.remove("./simple.exe")
                    compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", "simple.cpp"]
                    if run_cmd(compile_cmd):
                        if run_cmd(["./simple.exe"]) == "Hello":
                            my_compiler = ["Windows", "Visual Studio 2019", ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
                
        elif my_platform == "Linux":
            # Try GCC
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["g++", "simple.cpp", "-o", "simple"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Linux", "GCC (g++)", ["g++", f"{filename_base}.cpp", "-o", f"{filename_base}" ]]
    
            # Try Clang
            if not my_compiler:
                if os.path.isfile("./simple"):
                    os.remove("./simple")
                compile_cmd = ["clang++", "simple.cpp", "-o", "simple"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple"]) == "Hello":
                        my_compiler = ["Linux", "Clang++", ["clang++", f"{filename_base}.cpp", "-o", f"{filename_base}"]]
        
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    
        elif my_platform == "Darwin":
            # Mac with optimized settings
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "simple", "simple.cpp"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Macintosh", "Clang++", ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", f"{filename_base}", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    except:
        my_compiler=[my_platform, "Unavailable", []]
        
    if my_compiler:
        return my_compiler
    else:
        return ["Unknown", "Unavailable", []]

In [ ]:
# Code execution functions

def execute_python(code):
    """Execute Python code and capture output"""
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

def execute_cpp(code):
    """Compile and execute C++ code"""
    write_output(code)
    compiler_info = c_compiler_cmd("optimized")
    
    if compiler_info[1] == "Unavailable":
        return "No C++ compiler available on this system"
    
    try:
        # Compile
        compile_result = subprocess.run(compiler_info[2], check=True, text=True, capture_output=True)
        
        # Run
        if platform.system() == "Windows":
            run_cmd = ["./optimized.exe"]
        else:
            run_cmd = ["./optimized"]
            
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"Compilation/execution error:\n{e.stderr}"

In [ ]:
# Streaming functions for real-time UI updates

def stream_gemini(python):
    """Stream Gemini responses for real-time UI"""
    model = genai.GenerativeModel(GEMINI_MODEL)
    prompt = f"{system_message}\n\n{user_prompt_for(python)}"
    
    response = model.generate_content(
        prompt,
        stream=True,
        generation_config=genai.types.GenerationConfig(
            temperature=0.1,
            max_output_tokens=2000,
        )
    )
    
    reply = ""
    for chunk in response:
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```cpp\n','').replace('```','')

def stream_ollama(python, model_name):
    """Stream Ollama responses for real-time UI via direct request"""
    # Assuming OLLAMA_BASE_URL is defined as http://localhost:11434
    import requests
    url = "http://localhost:11434/api/generate"
    prompt = f"{system_message}\n\n{user_prompt_for(python)}"
    
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": True,
        "options": {"temperature": 0.1}
    }
    
    reply = ""
    try:
        response = requests.post(url, json=payload, stream=True)
        for line in response.iter_lines():
            if line:
                chunk = json.loads(line)
                if 'response' in chunk:
                    reply += chunk['response']
                    yield reply.replace('```cpp\n','').replace('```','')
    except Exception as e:
        yield f"Ollama local error: {e}"


In [ ]:
# Universal optimization function for all models

def optimize_code(python, model_choice):
    """Universal code optimization function supporting Gemini and Ollama"""
    if model_choice == "Gemini":
        result = stream_gemini(python)
    elif model_choice == "Ollama-Gemma":
        # Using the LOCAL_MODEL defined in initialization
        result = stream_ollama(python, LOCAL_MODEL)
    else:
        yield f"Unknown model: {model_choice}"
        return
        
    for stream_so_far in result:
        yield stream_so_far


In [ ]:
# Sample program selector

def select_sample_program(sample_program):
    """Select pre-defined sample programs"""
    if sample_program == "pi":
        return pi_calculation
    elif sample_program == "complex":
        return python_complex
    else:
        return "# Type your Python program here\nprint('Hello World')"

In [ ]:
# Auto-detect compiler and display system info

compiler_info = c_compiler_cmd("optimized")
print(f"Platform: {compiler_info[0]}")
print(f"Compiler: {compiler_info[1]}")
if compiler_info[2]:
    print(f"Compile command: {' '.join(compiler_info[2])}")
else:
    print("No C++ compiler detected")

In [ ]:
# Basic Gradio interface

css = """
.python {background-color: #306998; color: white;}
.cpp {background-color: #005500; color: white;}
.info {background-color: #f0f0f0; padding: 10px; border-radius: 5px;}
"""

with gr.Blocks(css=css, title="Week 4 - Code Generator") as basic_ui:
    gr.Markdown("## 🧪 Lab 4.1: Python to C++ Code Generator")
    gr.Markdown("Convert Python code to optimized C++ using Gemini 1.5 Flash or Local Gemma 2")
    
    with gr.Row():
        python_input = gr.Textbox(
            label="Python code:", 
            value=pi_calculation, 
            lines=10,
            classes=["python"]
        )
        cpp_output = gr.Textbox(
            label="C++ code:", 
            lines=10,
            classes=["cpp"]
        )
    
    with gr.Row():
        model_select = gr.Dropdown(
            ["Gemini", "Ollama-Gemma"], 
            label="Select model", 
            value="Gemini"
        )
        convert_btn = gr.Button("Convert to C++", variant="primary")
    
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++", interactive=(compiler_info[1] != "Unavailable"))
    
    with gr.Row():
        python_result = gr.TextArea(label="Python result:", classes=["python"])
        cpp_result = gr.TextArea(label="C++ result:", classes=["cpp"])

    convert_btn.click(optimize_code, inputs=[python_input, model_select], outputs=[cpp_output])
    python_run.click(execute_python, inputs=[python_input], outputs=[python_result])
    cpp_run.click(execute_cpp, inputs=[cpp_output], outputs=[cpp_result])

print("Basic Gradio interface ready.")


In [ ]:
# Launch basic interface

basic_ui.launch(inbrowser=True, share=False)

##  Lab 4.2: Advanced Code Optimizer (System Prompting)

**Aim**: To build a sophisticated interface to optimize algorithms dynamically using specialized system prompts and cross-platform performance benchmarking.

**Explanation**:
This lab expands on basic generation by incorporating:
1. **Cross-Platform Compilation Logic**: Automatic detection of system compilers.
2. **System Prompting for Performance**: Forcing the model to prioritize numerical parity and speed.
3. **Real-time Execution Benchmarking**: Comparing Python vs C++ execution times directly.

In [ ]:
# Advanced Gradio interface with sample programs

with gr.Blocks(css=css, title="Week 4 - Advanced Code Optimizer") as advanced_ui:
    gr.Markdown("## 🧪 Lab 4.2: Advanced Code Optimizer")
    gr.Markdown("Full-featured interface with sample programs and performance benchmarking")
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### System Information", classes=["info"])
            gr.Markdown(f"**Platform:** {compiler_info[0]}")
            gr.Markdown(f"**Compiler:** {compiler_info[1]}")
            
            sample_program = gr.Radio(
                ["pi", "complex", "custom"], 
                label="Sample program", 
                value="pi"
            )
            
            model_select = gr.Dropdown(
                ["Gemini", "Ollama-Gemma"], 
                label="Select AI model", 
                value="Gemini"
            )
            
        with gr.Column(scale=2):
            python_input = gr.Textbox(
                label="Python code:", 
                value=pi_calculation, 
                lines=15,
                classes=["python"]
            )
            
        with gr.Column(scale=2):
            cpp_output = gr.Textbox(
                label="Generated C++ code:", 
                lines=15,
                classes=["cpp"]
            )
    
    with gr.Row():
        convert_btn = gr.Button("🔄 Convert to C++", variant="primary", size="lg")
        
    with gr.Row():
        python_run = gr.Button("🐍 Run Python")
        if compiler_info[1] != "Unavailable":
            cpp_run = gr.Button("⚡ Run C++")
        else:
            cpp_run = gr.Button("❌ No C++ Compiler", interactive=False)
    
    with gr.Row():
        python_result = gr.TextArea(
            label="Python execution result:", 
            classes=["python"],
            max_lines=10
        )
        cpp_result = gr.TextArea(
            label="C++ execution result:", 
            classes=["cpp"],
            max_lines=10
        )

    # Event handlers
    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python_input])
    convert_btn.click(optimize_code, inputs=[python_input, model_select], outputs=[cpp_output])
    python_run.click(execute_python, inputs=[python_input], outputs=[python_result])
    cpp_run.click(execute_cpp, inputs=[cpp_output], outputs=[cpp_result])

print("Advanced Gradio interface ready.")


In [ ]:
# Launch advanced interface

advanced_ui.launch(inbrowser=True, share=False)

# Performance Testing

Test and compare performance between Python and C++ implementations:

In [ ]:
# Performance comparison function

def performance_test():
    """Run performance comparison between Python and C++ versions"""
    print("=== Performance Comparison ===")
    
    # Test Python performance
    print("\n1. Testing Python PI Calculation:")
    python_result = execute_python(pi_calculation)
    print(python_result)
    
    # Test Python complex algorithm
    print("\n2. Testing Python Complex Algorithm:")
    python_complex_result = execute_python(python_complex)
    print(python_complex_result)
    
    # Generate and test C++ version
    print("\n3. Generating C++ with Gemini:")
    try:
        cpp_code = optimize_gemini(python_complex)
        print("\n4. Testing C++ version:")
        cpp_result = execute_cpp(cpp_code)
        print(cpp_result)
    except Exception as e:
        print(f"Error in C++ generation/execution: {e}")

# Run performance test
# performance_test()  # Uncomment to run
print("Performance test function ready. Uncomment the last line to run it.")

---
##  Reference: Generated C++ Examples

Below are examples of what the model typically generates for the benchmark algorithms.

###  simple.cpp
```cpp
#include <iostream>

int main() {
std::cout << "Hello";
return 0;
}
```

###  optimized.cpp (Complex Algorithm)
```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>
#include <iomanip>

class LCG {
private:
uint64_t value;
const uint64_t a = 1664525;
const uint64_t c = 1013904223;
const uint64_t m = 1ULL << 32;

public:
LCG(uint64_t seed) : value(seed) {}

uint64_t next() {
value = (a * value + c) % m;
return value;
}
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
LCG lcg(seed);
std::vector<int> random_numbers(n);
for (int i = 0; i < n; ++i) {
random_numbers[i] = static_cast<int>(lcg.next() % (max_val - min_val + 1) + min_val);
}

int64_t max_sum = std::numeric_limits<int64_t>::min();
int64_t current_sum = 0;
for (int i = 0; i < n; ++i) {
current_sum = std::max(static_cast<int64_t>(random_numbers[i]), current_sum + random_numbers[i]);
max_sum = std::max(max_sum, current_sum);
}
return max_sum;
}

int64_t total_max_subarray_sum(int n, uint64_t initial_seed, int min_val, int max_val) {
int64_t total_sum = 0;
LCG lcg(initial_seed);
for (int i = 0; i < 20; ++i) {
uint64_t seed = lcg.next();
total_sum += max_subarray_sum(n, seed, min_val, max_val);
}
return total_sum;
}

int main() {
int n = 10000;
uint64_t initial_seed = 42;
int min_val = -10;
int max_val = 10;

auto start_time = std::chrono::high_resolution_clock::now();
int64_t result = total_max_subarray_sum(n, initial_seed, min_val, max_val);
auto end_time = std::chrono::high_resolution_clock::now();

auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);

std::cout << "Total Maximum Subarray Sum (20 runs): " << result << std::endl;
std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << duration.count() / 1e6 << " seconds" << std::endl;

return 0;
}
```
---

In [ ]:
# Final summary and completion

print("="*60)
print("WEEK 4 COMPILED NOTEBOOK - SETUP COMPLETE")
print("="*60)
print()
print("✅ All code from day3.ipynb and day4.ipynb has been compiled")
print("✅ Updated to use Gemini 1.5 Flash")
print("✅ Added Ollama support with Gemma 2:2b model")
print("✅ Cross-platform C++ compilation support included")
print("✅ Both basic and advanced Gradio interfaces ready")
print("✅ Performance testing utilities included")
print("✅ All original C++ examples preserved")
print()
print("🚀 Ready to generate high-performance C++ code from Python!")
print("📊 Use the Gradio interfaces above to test the functionality")
print()
print("Models configured:")
print(f"  • Gemini: {GEMINI_MODEL}")
print(f"  • Ollama Gemma: {LOCAL_MODEL}")
print()
print(f"System: {compiler_info[0]} with {compiler_info[1]} compiler")
print("="*60)


---

##  Instructor's Evaluation & Lab Summary

###  Assessment Criteria
1. **Technical Implementation**: Adherence to the lab objectives and code functionality.
2. **Logic & Reasoning**: Clarity in the explanation of the underlying AI principles.
3. **Best Practices**: Use of secure environment variables and structured prompts.

**Lab Completion Status: Verified**
**Focus Area**: Language Modelling & Deep Learning Systems.